> ⚠️ **작업 중 (Work in Progress)**: 이 노트북은 현재 개발 중입니다. 일부 코드가 불완전하거나 변경될 수 있습니다.

## 📋 목차

- [에이전트 개요](#에이전트-개요)
- [ModelRouterAgent 생성](#modelrouteragent-생성)
- [FileSearchAgent 생성](#filesearchagent-생성)
- [WebSearchAgent 생성](#websearchagent-생성)
- [에이전트 배포 및 호출](#에이전트-배포-및-호출)

## 🎯 학습 목표

- Microsoft Foundry 에이전트의 핵심 개념 이해
- Model Router 기반 에이전트 구축
- File Search 기능을 활용한 문서 기반 에이전트 생성
- Web Search 기능을 활용한 실시간 정보 검색 에이전트 생성
- 에이전트 배포 및 프로그래매틱 호출 방법 학습

## ⏱️ 예상 소요 시간

약 30분

## 환경 설정

먼저 프로젝트 엔드포인트를 설정합니다.

In [ ]:
# 환경 변수 로드
import json
import os
import subprocess

# PATH 환경변수 설정 (Azure CLI를 찾을 수 있도록)
possible_paths = [
    "/opt/homebrew/bin",  # macOS (Apple Silicon)
    "/usr/local/bin",     # macOS (Intel) / Linux
    "/usr/bin",           # Linux / GitHub Codespaces
    "/home/linuxbrew/.linuxbrew/bin"  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
except:
    pass

paths_to_add = []
if az_path and az_path not in os.environ.get("PATH", ""):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get("PATH", ""):
            paths_to_add.append(path)

if paths_to_add:
    new_path = ":".join(paths_to_add) + ":" + os.environ.get("PATH", "")
    os.environ["PATH"] = new_path

# 이전 노트북에서 저장한 설정 파일 로드
config_file = ".foundry_config.json"
try:
    with open(config_file, 'r') as f:
        config = json.load(f)
    
    # 환경 변수 설정
    FOUNDRY_NAME = config.get("FOUNDRY_NAME")
    RESOURCE_GROUP = config.get("RESOURCE_GROUP")
    LOCATION = config.get("LOCATION")
    TENANT_ID = config.get("TENANT_ID")
    PROJECT_NAME = config.get("PROJECT_NAME", "proj-default")
    PROJECT_ENDPOINT = config.get("FOUNDRY_ENDPOINT")
    
    # 환경 변수로도 설정 (다른 도구들이 사용할 수 있도록)
    os.environ["FOUNDRY_NAME"] = FOUNDRY_NAME
    os.environ["LOCATION"] = LOCATION
    os.environ["RESOURCE_GROUP"] = RESOURCE_GROUP
    os.environ["AZURE_SUBSCRIPTION_ID"] = config.get("AZURE_SUBSCRIPTION_ID", "")
    os.environ["_ENDPOINT"] = PROJECT_ENDPOINT
    
    print(f"✅ 설정 파일 '{config_file}'에서 환경 변수를 로드했습니다.")
    print(f"\n📌 Foundry Name: {FOUNDRY_NAME}")
    print(f"📌 Resource Group: {RESOURCE_GROUP}")
    print(f"📌 Location: {LOCATION}")
    print(f"📌 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")
    
except FileNotFoundError:
    print(f"⚠️ '{config_file}' 파일을 찾을 수 없습니다.")
    print("💡 01-setup.ipynb를 먼저 실행하여 환경을 설정하세요.")
    raise

# 필수 패키지 설치
%pip install -q azure-ai-projects==2.0.0b2 azure-identity

from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from azure.identity import DefaultAzureCredential

print(f"\n💡 사용할 프로젝트 엔드포인트: {PROJECT_ENDPOINT}")

## ModelRouterAgent 생성

Model Router를 활용하여 지능적으로 모델을 선택하는 에이전트를 만듭니다.

**Agent 구성:**
- **Model**: model-router (비용/품질/성능 자동 최적화)
- **Instructions**: 질문 답변 에이전트
- **Tools**: 없음 (기본 대화)

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=PROJECT_ENDPOINT, credential=credential)

# ModelRouterAgent 생성
ROUTER_INSTRUCTIONS = """당신은 질문에 답변하는 에이전트입니다.
요청의 복잡도와 요구사항에 따라 가장 적절한 모델을 사용하세요.
항상 명확하고 정확하며 도움이 되는 답변을 제공하세요."""

try:
    definition = PromptAgentDefinition(
        model="model-router",
        instructions=ROUTER_INSTRUCTIONS
    )
    
    agent_router = client.agents.create(
        name="ModelRouterAgent",
        definition=definition
    )
    
    print(f"✅ ModelRouterAgent 생성 완료!")
    print(f"   ID: {agent_router.id}")
    print(f"   Name: {agent_router.name}")
    print(f"   Model: {definition.model}")
    
except Exception as e:
    print(f"⚠️ 에이전트 생성 실패: {e}")
    print("\n💡 해결 방법:")
    print("   1. Portal에서 'model-router' 모델이 배포되었는지 확인")
    print("   2. 모델 이름이 정확한지 확인 (대소문자 구분)")

## FileSearchAgent 생성

파일 검색 기능을 활용하여 업로드된 문서에서 정보를 찾는 에이전트입니다.

**Agent 구성:**
- **Model**: gpt-5.1
- **Tools**: file_search (파일 내용 검색)
- **Files**: knowledge-base.json 업로드

**⚠️ 주의**: 파일 업로드는 Azure Portal(https://ai.azure.com)에서 더 편리합니다.
- Build > Agents > Create agent > Tools > File Search > Upload files

In [ ]:
# FileSearchAgent 생성

FILE_SEARCH_INSTRUCTIONS = """너는 Tools에 등록된 File search 기반으로 답변하는 에이전트입니다.

중요 규칙:
1. 반드시 업로드된 파일의 내용을 기반으로만 답변하세요
2. 파일에 없는 정보는 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 답변하세요
3. 답변 시 출처 파일명을 언급하세요
4. 정확한 인용을 사용하세요"""

try:
    # FileSearchAgent 생성 (tools 없이 먼저 생성)
    definition = PromptAgentDefinition(
        model="gpt-5.1",
        instructions=FILE_SEARCH_INSTRUCTIONS
    )
    
    agent_filesearch = client.agents.create(
        name="FileSearchAgent",
        definition=definition
    )
    
    print(f"✅ FileSearchAgent 생성 완료!")
    print(f"   ID: {agent_filesearch.id}")
    print(f"   Name: {agent_filesearch.name}")
    print(f"   Model: {definition.model}")
    print(f"\n📋 다음 단계: Azure Portal에서 File Search 도구 추가")
    print(f"   1. Azure Portal (https://ai.azure.com) 접속")
    print(f"   2. Build > Agents > 'FileSearchAgent' 선택")
    print(f"   3. Tools 섹션에서 'File Search' 도구 추가")
    print(f"   4. knowledge-base.json 파일 업로드")
    print(f"   5. 업로드 후 에이전트가 파일 내용을 검색할 수 있습니다")
    
except Exception as e:
    print(f"⚠️ 에이전트 생성 실패: {e}")
    print("\n💡 해결 방법:")
    print("   1. 'gpt-5.1' 모델이 배포되었는지 확인")
    print("   2. Portal에서 모델 이름 확인 (대소문자 구분)")

## WebSearchAgent 생성

웹 검색 기능을 활용하여 실시간 정보를 제공하는 에이전트입니다.

**Agent 구성:**
- **Model**: gpt-4.1
- **Tools**: web_search (웹 검색)
- **기능**: 최신 뉴스, 날씨, 주식 정보 등

In [ ]:
# WebSearchAgent 생성

WEB_SEARCH_INSTRUCTIONS = """너는 Tools에 등록된 Web search 기반으로 답변하는 에이전트입니다.

중요 규칙:
1. 최신 정보가 필요한 질문에는 반드시 웹 검색을 사용하세요
2. 검색 결과를 기반으로 정확하고 최신의 정보를 제공하세요
3. 답변 시 출처 URL을 포함하세요
4. 여러 출처의 정보를 종합하여 균형잡힌 답변을 제공하세요
5. 검색 결과가 불충분하면 추가 검색을 수행하세요"""

try:
    definition = PromptAgentDefinition(
        model="gpt-4.1",
        instructions=WEB_SEARCH_INSTRUCTIONS,
        tools=[{"type": "web_search"}]
    )
    
    agent_websearch = client.agents.create(
        name="WebSearchAgent",
        definition=definition
    )
    
    print(f"✅ WebSearchAgent 생성 완료!")
    print(f"   ID: {agent_websearch.id}")
    print(f"   Name: {agent_websearch.name}")
    print(f"   Model: {definition.model}")
    print(f"   Tools: Web Search")
    
except Exception as e:
    print(f"⚠️ 에이전트 생성 실패: {e}")
    print("\n💡 해결 방법:")
    print("   1. 'gpt-4.1' 모델이 배포되었는지 확인")
    print("   2. Web Search가 프로젝트에서 활성화되었는지 확인")

### 생성된 에이전트 목록 확인

## 생성된 에이전트 확인

In [ ]:
# 모든 에이전트 목록 조회
agents = client.agents.list()

print("=" * 80)
print("생성된 에이전트 목록")
print("=" * 80)

for agent in agents:
    print(f"\n📌 {agent.name}")
    print(f"   ID: {agent.id}")
    
    # versions에서 정보 추출
    if 'latest' in agent.versions:
        latest = agent.versions['latest']
        definition = latest.get('definition', {})
        
        model = definition.get('model', 'N/A')
        print(f"   Model: {model}")
        
        tools = definition.get('tools', [])
        if tools:
            tool_types = [t.get('type', 'unknown') if isinstance(t, dict) else str(t) for t in tools]
            print(f"   Tools: {', '.join(tool_types)}")
        else:
            print(f"   Tools: None")

print("\n✅ 포털 확인: https://ai.azure.com > Build > Agents")

## 에이전트 테스트 (선택)

In [ ]:
# 에이전트 객체 구조 확인 (디버깅용)
print("🔍 agent_router 객체 디버깅:")
print(f"Type: {type(agent_router)}")
print(f"\nAttributes:")
for attr in dir(agent_router):
    if not attr.startswith('_'):
        try:
            value = getattr(agent_router, attr)
            if not callable(value):
                print(f"  {attr}: {value}")
        except:
            pass

print(f"\n📋 Raw object:")
print(agent_router)

In [ ]:
# 간단한 에이전트 테스트 (선택사항)
# ModelRouterAgent와 대화

try:
    print("⏳ ModelRouterAgent 실행 중...")
    print(f"📍 Agent ID: {agent_router.id}")
    print(f"📍 Agent Name: {agent_router.name}")
    
    # SDK v2 - project client를 통해 OpenAI client 획득
    openai_client = client.get_openai_client()
    
    # Conversation 생성
    conversation = openai_client.conversations.create()
    print(f"💬 Conversation ID: {conversation.id}")
    
    # Responses API 호출
    response = openai_client.responses.create(
        conversation=conversation.id,
        extra_body={"agent": {"name": agent_router.name, "type": "agent_reference"}},
        input="Python에서 리스트를 정렬하는 방법을 알려주세요."
    )
    
    print("\n" + "=" * 80)
    print("🤖 ModelRouterAgent 응답:")
    print("=" * 80)
    print(response.output_text)
    
    # Conversation 정리
    openai_client.conversations.delete(conversation_id=conversation.id)
    print("\n✅ 테스트 성공!")
    
except NameError:
    print("⚠️ agent_router 또는 client가 정의되지 않았습니다.")
    print("💡 위의 환경 설정 및 ModelRouterAgent 생성 셀을 먼저 실행하세요.")
    
except Exception as e:
    print(f"⚠️ 테스트 실패: {e}")
    import traceback
    print(f"\n상세 오류:\n{traceback.format_exc()}")

In [ ]:
# WebSearchAgent 테스트 (선택사항)
# 최신 정보 검색

try:
    print("⏳ WebSearchAgent 실행 중 (웹 검색 중...)")
    print(f"📍 Agent ID: {agent_websearch.id}")
    print(f"📍 Agent Name: {agent_websearch.name}")
    
    # SDK v2 - project client를 통해 OpenAI client 획듍
    openai_client = client.get_openai_client()
    
    # Conversation 생성
    conversation = openai_client.conversations.create()
    print(f"💬 Conversation ID: {conversation.id}")
    
    # Responses API 호출
    response = openai_client.responses.create(
        conversation=conversation.id,
        extra_body={"agent": {"name": agent_websearch.name, "type": "agent_reference"}},
        input="오늘 서울 날씨는 어떤가요?"
    )
    
    print("\n" + "=" * 80)
    print("🌐 WebSearchAgent 응답:")
    print("=" * 80)
    print(response.output_text)
    
    # Conversation 정리
    openai_client.conversations.delete(conversation_id=conversation.id)
    print("\n✅ 웹 검색 도구가 실시간 정보를 가져왔습니다!")
    
except NameError:
    print("⚠️ agent_websearch 또는 client가 정의되지 않았습니다.")
    print("💡 위의 환경 설정 및 WebSearchAgent 생성 셀을 먼저 실행하세요.")
    
except Exception as e:
    print(f"⚠️ 테스트 실패: {e}")
    import traceback
    print(f"\n상세 오류:\n{traceback.format_exc()}")

### ✅ 확인 사항

- 에이전트가 성공적으로 게시되었는지 확인
- Python 스크립트가 에러 없이 실행되는지 확인
- 응답이 예상대로 반환되는지 확인

## 📚 추가 리소스

- [Microsoft Foundry Agents 개요](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/overview?view=foundry)
- [Agent SDK 문서](https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/develop/sdk-overview?view=foundry&pivots=programming-language-python)
- [File Search 가이드](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/file-search?view=foundry&pivots=python)
- [Web Search 통합](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/web-search?view=foundry&pivots=python)

## 다음 단계

다양한 에이전트를 만들어보았습니다! 이제 Foundry IQ를 사용하여 고급 지식 기반을 구축해봅시다:

➡️ **[04. Foundry IQ](./04-foundry-iq.ipynb)**: AI Search와 Blob Storage를 활용한 지식 기반 구축을 학습합니다.